# Acid
```
RM02/0001
RM02/0002
```

In [7]:
%%capture
!pip install fredapi # To install additional package for Datalab

In [8]:
import preprocessor as pre
import forecastor as fc
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

## Preparations
# Import data
gas_df = pre.get_Fred_data('PNGASEUUSDM',2014,2024)
wheat_df = pre.get_Fred_data('PWHEAMTUSDM',2014,2024)
ammonia_df = pre.get_Fred_data('WPU0652013A',2014,2024)
elec_df = pre.clean_elec_csv('Data_flat_files/ELECTRICITY_03_2024.csv',2014,2024)

df = pre.clean_pred_price_evo_csv("Data_flat_files/Dataset_Future_Predicting_Price_Evolutions_202403.csv",2014,2023)

target = 'acid'.lower()

RM_codes = ['RM01/0001','RM01/0004','RM01/0006','RM01/0007']

external_drivers = {
    "PNGASEUUSDM": gas_df,
    "PWHEAMTUSDM": wheat_df,
    "WPU0652013A": ammonia_df,
    "Electricity": elec_df
}

test_periods = [
    ('2020-01-01', '2020-07-01'),
    ('2020-07-01', '2021-01-01'),
    ('2021-01-01', '2021-07-01'),
    ('2021-07-01', '2022-01-01'),
    ('2022-01-01', '2022-07-01'),
    ('2022-07-01', '2023-01-01'),
    ('2023-01-01', '2023-07-01'),
    ('2023-07-01', '2024-01-01')
]

lags = [1,3,6]

In [3]:
# Impute raw data of target variables 
imputed_df, missing = pre.impute_pred_price_evo_csv(df)

# Feature engineering
dummy_df = pre.get_dummies_and_average_price(imputed_df,target,*RM_codes)
feature_df = pre.generate_features(1,12,dummy_df,missing,*RM_codes, **external_drivers)

if type(feature_df.Time) != "datetime64":
    feature_df['Time'] = pd.to_datetime(feature_df['Time'])

assert feature_df['Time'].dtype == "datetime64[ns]" , "df[Time] is not dataetime64."

feature_df = feature_df[feature_df.Year >= 2016]

In [4]:
# Persistent Naive
for code in RM_codes:
    for lag in lags:
        for period in test_periods:
            mape_values = list()
            result = fc.persistence_Naive_MAPE(feature_df,code,lag,period)
            mape_values.append(result)
            
        average_mape = np.mean(mape_values) 
        print(f"{target} {code}, {lag}-month lag, Naive, average MAPE: {average_mape:.3f}")

acid RM01/0001, 1-month lag, Naive, average MAPE: 5.727
acid RM01/0001, 3-month lag, Naive, average MAPE: 18.198
acid RM01/0001, 6-month lag, Naive, average MAPE: 40.799
acid RM01/0004, 1-month lag, Naive, average MAPE: 3.266
acid RM01/0004, 3-month lag, Naive, average MAPE: 7.921
acid RM01/0004, 6-month lag, Naive, average MAPE: 9.182
acid RM01/0006, 1-month lag, Naive, average MAPE: 16.130
acid RM01/0006, 3-month lag, Naive, average MAPE: 17.225
acid RM01/0006, 6-month lag, Naive, average MAPE: 53.719
acid RM01/0007, 1-month lag, Naive, average MAPE: 4.941
acid RM01/0007, 3-month lag, Naive, average MAPE: 13.287
acid RM01/0007, 6-month lag, Naive, average MAPE: 32.057


In [5]:
# Lasso with autoregression features only
for code in RM_codes:
    for lag in lags:
        for period in test_periods:
            mape_values = list()
            result = fc.train_model_AR(feature_df,code,lag,period)
            mape_values.append(result)
            
        average_mape = np.mean(mape_values) 
        print(f"{target} {code}, {lag}-month lag, AR, average MAPE: {average_mape:.3f}")

acid RM01/0001, 1-month lag, AR, average MAPE: 6.063
acid RM01/0001, 3-month lag, AR, average MAPE: 7.107
acid RM01/0001, 6-month lag, AR, average MAPE: 19.940
acid RM01/0004, 1-month lag, AR, average MAPE: 4.299
acid RM01/0004, 3-month lag, AR, average MAPE: 16.607
acid RM01/0004, 6-month lag, AR, average MAPE: 47.180
acid RM01/0006, 1-month lag, AR, average MAPE: 15.252
acid RM01/0006, 3-month lag, AR, average MAPE: 19.810
acid RM01/0006, 6-month lag, AR, average MAPE: 21.749
acid RM01/0007, 1-month lag, AR, average MAPE: 11.228
acid RM01/0007, 3-month lag, AR, average MAPE: 20.335
acid RM01/0007, 6-month lag, AR, average MAPE: 22.825


In [6]:
# Lasso with autoregression features and external price drivers
for code in RM_codes:
    for lag in lags:
        mape_values = list()
        for period in test_periods:
            result = fc.train_model_all_features(feature_df,code,lag,period)
            mape_values.append(result)
            
        average_mape = np.mean(mape_values) 
        print(f"{target} {code}, {lag}-month lag, all features, average MAPE: {average_mape:.3f}")

acid RM01/0001, 1-month lag, all features, average MAPE: 5.874
acid RM01/0001, 3-month lag, all features, average MAPE: 11.453
acid RM01/0001, 6-month lag, all features, average MAPE: 21.402
acid RM01/0004, 1-month lag, all features, average MAPE: 12.456
acid RM01/0004, 3-month lag, all features, average MAPE: 21.056
acid RM01/0004, 6-month lag, all features, average MAPE: 33.762
acid RM01/0006, 1-month lag, all features, average MAPE: 15.956
acid RM01/0006, 3-month lag, all features, average MAPE: 16.906
acid RM01/0006, 6-month lag, all features, average MAPE: 26.773
acid RM01/0007, 1-month lag, all features, average MAPE: 23.094
acid RM01/0007, 3-month lag, all features, average MAPE: 26.471
acid RM01/0007, 6-month lag, all features, average MAPE: 28.916
